In [40]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\Jon\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache


NASA Mars News

Scrape the Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [41]:

# Load the page
url = 'https://redplanetscience.com/'
browser.visit(url)

# Scrape into Soup
html = browser.html
soup = bs(html, 'html.parser')

# get the first list_date (if needed) and retain content_title and article_teaser_body inside of list_text as they are in chronological order with the latest being first
# MarsNews_latest_date = soup.find('div', class_='list_date').get_text()
MarsNews_latest_Title = soup.find('div', class_='content_title').get_text()
MarsNews_latest_ArticleBody = soup.find('div', class_='article_teaser_body').get_text()
print(MarsNews_latest_Title + '\n' + MarsNews_latest_ArticleBody)


NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities
Starting July 27, news activities will cover everything from mission engineering and science to returning samples from Mars to, of course, the launch itself.


JPL Mars Space Images - Featured Image

Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.


In [42]:

# Load the page
url = 'https://spaceimages-mars.com/'
browser.visit(url)

# Scrape into Soup
html = browser.html
soup = bs(html, 'html.parser')

# Make sure to find the image url to the full size .jpg image.
btn = soup.find('button', text=' FULL IMAGE')
if btn.parent.name == 'a':
    a = btn.parent

if a:
    featured_image_url = a['href']

# Make sure to save a complete url string for this image.
featured_image_url = url + featured_image_url
featured_image_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

Mars Facts

Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

In [43]:
# Use Pandas to convert the data to a HTML table string.

# scrape the site and gather the html for the table
url = 'https://galaxyfacts-mars.com/'
table = pd.read_html(url, match='Equatorial Diameter')

# convert to dataframe
df = table[0]

# render out to html table string
html_tbl = df.to_html()
html_tbl = html_tbl.replace('\n', '')
html_tbl

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>0</th>      <th>1</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 ( Phobos &amp; Deimos )</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    

Mars Hemispheres

Visit the astrogeology site here to obtain high resolution images for each of Mar's hemispheres.


In [44]:
hemisphere_image_urls = []

# load initial page
url = 'https://marshemispheres.com/'
browser.visit(url)

# Scrape into Soup
html = browser.html
soup = bs(html, 'html.parser')

# Second browser to find full res image in sub pages
browser2 = Browser("chrome", **executable_path, headless=False)

# find the parent of the list of hemishpere items we want
container = soup.find('div', class_='results')

# get all of the hemispheres within
items = container.find_all('div', class_='item')

# loop through each hemishpere 
for item in items:
   
    # get the title 
    title = item.find('h3').get_text() 

    # launch second browser to sub page to find the full res image
    page = item.find('a', class_='product-item')['href']
    url2 = url + page   
    browser2.visit(url2)
    html2 = browser2.html
    soup2 = bs(html2, 'html.parser')

    img_url = url + soup2.find('a', text='Original')['href']

    # Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.
    hemishpere_dict = {
        "title":title, 
        "img_url":img_url
    }

    # Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.
    hemisphere_image_urls.append(hemishpere_dict)

browser2.quit()
browser.quit()

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/cerberus_enhanced.tif'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced.tif'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced.tif'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced.tif'}]